### ========================= Import Modules =========================

In [1]:
# import python libraries
import sys, os

# Import third party libraries
import pandas as pd

# Dynamically get the current working directory and append 'src' to it
# Not needed when using pip install GeeseTools
sys.path.append(os.path.join(os.getcwd(), "src"))
from DataPreProcessor import DataPreProcessor as dpp

from utils import train_models as tm
from utils import evaluation as eval
from utils import plot

### =========================== Load Data ===========================

In [2]:
# Creating object for DataPreProcessor Class
obj = dpp(pd.read_csv("heart.csv"), target=["age", "diagnosis"])

In [3]:
obj.display_all_features()

,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,diagnosis
0,1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,1
2,3,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,4,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,5,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0


### ========================= Summary of Data ========================

In [40]:
obj.unique_value_summary()

,Unique Values,Total Values,Percentage (%)
id,866,866,100.000000
age,50,866,5.773672
sex,2,866,0.230947
dataset,4,866,0.461894
cp,4,866,0.461894
trestbps,62,866,7.159353
chol,210,866,24.249423
fbs,2,866,0.230947
restecg,3,866,0.346420
thalch,120,866,13.856813


In [5]:
obj.missing_data_summary().head() 

,Variable,Missing Count,Missing Percentage
0,id,0,0.0%
1,age,0,0.0%
2,sex,0,0.0%
3,dataset,0,0.0%
4,cp,0,0.0%


### ========================= Pre-process Data =========================

In [6]:
# Pre-processing dataset initialized in obj
X_train, X_test, y_train, y_test = obj.pre_process()

### ======================== Transformation Logs =======================

In [7]:
obj.to_numeric_log_df.head()  

,Column Name,Original Value,Converted Value,Conversion Type
0,sex,Male,male,str to str
1,sex,Male,male,str to str
2,sex,Male,male,str to str
3,sex,Male,male,str to str
4,sex,Female,female,str to str


In [8]:
obj.dropped_features_log_df.head() 

,Variable,Missing Percentage
0,slope,33.59
1,ca,66.41
2,thal,52.83


In [9]:
obj.dropped_records_log_df.head()

,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,diagnosis
714,715,69,male,switzerland,asymptomatic,NaN,0.0,False,st-t abnormality,NaN,NaN,NaN,1
733,734,63,male,va long beach,non-anginal,NaN,0.0,False,lv hypertrophy,NaN,NaN,NaN,1
743,744,74,male,va long beach,non-anginal,NaN,0.0,False,normal,NaN,NaN,NaN,0
746,747,51,male,va long beach,asymptomatic,NaN,0.0,True,st-t abnormality,NaN,NaN,NaN,1
749,750,55,male,va long beach,non-anginal,NaN,228.0,False,st-t abnormality,NaN,NaN,NaN,1


In [10]:
obj.imputation_log_df.head()      

,Variable,Imputation Method,Significant Difference,Percentage Difference
0,id,None (No Missing Values),0,0.0
1,age,None (No Missing Values),0,0.0
2,trestbps,Mean,0,1.61
3,chol,Median,1,10.98
4,thalch,Mean,0,1.75


In [11]:
obj.encode_log_df.head()          

,Column Name,Original Unique Values,Encoding Method,Encoded Unique Values
0,sex,"[male, female]",Nominal,"[1.0, 0.0]"
1,dataset,"[cleveland, hungary, switzerland, va long beach]",Nominal,"[0.0, 1.0, 2.0, 3.0]"
2,cp,"[typical angina, asymptomatic, non-anginal, at...",Nominal,"[3.0, 0.0, 2.0, 1.0]"
3,fbs,"[True, False]",Nominal,"[1.0, 0.0]"
4,restecg,"[lv hypertrophy, normal, st-t abnormality]",Nominal,"[0.0, 1.0, 2.0]"


In [12]:
obj.transformation_log_df.head()  

,Column Name,Skewness Before,Kurtosis Before,Action Taken,Skewness After,Kurtosis After
0,id,0.072783,-1.119606,Yeo-Johnson Transformation,-0.260257,-0.959040
1,trestbps,0.213951,2.993188,Yeo-Johnson Transformation,0.277478,2.661651
2,chol,-0.636879,0.196727,Yeo-Johnson Transformation,-1.037892,0.089642
3,thalch,-0.211240,-0.476804,Yeo-Johnson Transformation,-0.054580,-0.596636
4,oldpeak,1.053796,1.169938,Log Transformation,0.468229,-1.159658


In [13]:
obj.scale_log_df.head()

,Column Name,Scaling Method,Original Min,Original Max,Scaled Min,Scaled Max
0,id,Standard,-2.174470,1.658970,-2.174470,1.658970
1,dataset,Standard,0.000000,3.000000,-1.051200,1.741251
2,cp,Standard,0.000000,3.000000,-0.811481,2.354026
3,trestbps,Standard,-6.688428,3.605130,-6.688428,3.605130
4,chol,Standard,-1.934231,2.765033,-1.934231,2.765033


### ========================= Train NN Model =========================

In [14]:
model, task_type, history = tm.train_model(X_train, y_train)

🔍 Regression problem detected.


In [22]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

# Input layer
input_layer = Input(shape=(X_train.shape[1],))  # Replace with your feature size

# Shared base
shared = Dense(64, activation='relu')(input_layer)
shared = Dense(32, activation='relu')(shared)

# Output 1: Binary classification (diagnosis)
diagnosis_output = Dense(1, activation='sigmoid', name='diagnosis')(shared)

# Output 2: Regression (age)
age_output = Dense(1, name='age')(shared)

# Full model
model = Model(inputs=input_layer, outputs=[diagnosis_output, age_output])

# Compile with different loss functions for each task
model.compile(
    optimizer='adam',
    loss={'diagnosis': 'binary_crossentropy', 'age': 'mse'},
    metrics={'diagnosis': 'accuracy', 'age': 'mae'}
)


In [26]:
y_train_diagnosis = y_train.iloc[:, 0].values.reshape(-1, 1)
y_train_age = y_train.iloc[:, 1].values.reshape(-1, 1)

y_test_diagnosis = y_test.iloc[:, 0].values.reshape(-1, 1)
y_test_age = y_test.iloc[:, 1].values.reshape(-1, 1)



In [29]:
model.fit(X_train, {'diagnosis': y_train_diagnosis, 'age': y_train_age}, epochs=10, batch_size=32, verbose=0)

In [37]:
results = model.evaluate(X_test, {
    'diagnosis': y_test_diagnosis,
    'age': y_test_age
}, return_dict=True)


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - age_loss: 1.9170 - age_mae: 1.0806 - diagnosis_accuracy: 0.0000e+00 - diagnosis_loss: -84252.5781 - loss: -84620.7734 


In [38]:
import pandas as pd
from IPython.display import display

results_df = pd.DataFrame(results.items(), columns=["Metric", "Value"])
results_df["Value"] = results_df["Value"].round(4)
display(results_df)


,Metric,Value
0,age_loss,1.8798
1,age_mae,1.0829
2,diagnosis_accuracy,0.0000
3,diagnosis_loss,-85075.5703
4,loss,-86369.0703


### ========================= Evaluate Model =========================

In [15]:
metric, y_pred = eval.evaluate_model(model, X_test, y_test, task_type)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 


InvalidIndexError: (slice(None, None, None), 0)

Plotting Confusion Matrix (Classification)


In [ ]:
plot.plot_classification_outputs(y_test, y_pred)